# Tavily 검색 Tool을 활용한 LangChain Agent

In [15]:
# 필요한 패키지 설치
%pip install langchain langchain-openai langchain-community tavily-python python-dotenv -q

Note: you may need to restart the kernel to use updated packages.


In [16]:
import os
from pathlib import Path
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain import hub

# .env 파일 경로 설정
env_path = Path.cwd() / ".env"
if not env_path.exists():
    env_path = Path("uuyeong/.env")
    if not env_path.exists():
        env_path = Path(__file__).parent / ".env" if "__file__" in globals() else Path.cwd() / "uuyeong/.env"

# .env 파일에서 환경변수 로드
load_dotenv(dotenv_path=env_path)
print(f".env 파일 경로: {env_path.absolute()}")

# API 키 확인
if "TAVILY_API_KEY" not in os.environ or os.environ["TAVILY_API_KEY"] == "YOUR_KEY":
    print("경고: TAVILY_API_KEY 환경변수가 설정되지 않았습니다.")
    print("uuyeong/.env 파일에서 TAVILY_API_KEY를 설정해주세요.")
else:
    print("Tavily API 키가 설정되었습니다.")
    print(f"API 키 (처음 10자): {os.environ['TAVILY_API_KEY'][:10]}...")

if "OPENAI_API_KEY" not in os.environ or os.environ["OPENAI_API_KEY"] == "YOUR_KEY":
    print("경고: OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")
    print("uuyeong/.env 파일에서 OPENAI_API_KEY를 설정해주세요.")
else:
    print("OpenAI API 키가 설정되었습니다.")
    print(f"API 키 (처음 10자): {os.environ['OPENAI_API_KEY'][:10]}...")


.env 파일 경로: /Users/mac/Desktop/hateslop/hw11/3-Agent-3/uuyeong/.env
Tavily API 키가 설정되었습니다.
API 키 (처음 10자): tvly-dev-d...
OpenAI API 키가 설정되었습니다.
API 키 (처음 10자): sk-proj-x2...


## Tavily Tool을 LangChain Tool로 등록


In [17]:
# Tavily 검색 Tool 생성
tavily_tool = TavilySearchResults(
    api_key=os.environ.get("TAVILY_API_KEY", "YOUR_KEY"),
    max_results=5
)

print("Tavily 검색 Tool이 생성되었습니다.")
print(f"최대 검색 결과 수: {tavily_tool.max_results}")


Tavily 검색 Tool이 생성되었습니다.
최대 검색 결과 수: 5


## AgentExecutor로 연결

OpenAI LLM과 Tavily Tool을 연결하여 Agent를 생성


In [18]:
# LLM 초기화 (.env 파일에서 OPENAI_API_KEY 자동 로드됨)
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# Agent 생성
tools = [tavily_tool]
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("AgentExecutor가 생성되었습니다.")
print(f"사용 가능한 Tool: {[tool.name for tool in tools]}")


AgentExecutor가 생성되었습니다.
사용 가능한 Tool: ['tavily_search_results_json']


## 최신 정보 질문 실행

Tavily 검색을 통해 최신 정보를 검색하고 요약하는 예시들을 실행


### example 1 : 날씨 정보 검색


In [19]:
result1 = agent_executor.invoke({"input": "내일 서울 비 와?"})
print("\n" + "="*50)
print("최종 답변:")
print("="*50)
print(result1["output"])




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '서울 내일 날씨 예보'}`


[{'title': '서울 실시간 날씨 정보 | 서울 공식 관광정보 웹사이트', 'url': 'https://korean.visitseoul.net/weather', 'content': '| 항목 | 오늘 : 2025-12-01 | 내일 : 2025-12-02 | 모레 : 2025-12-03 |\n| 날씨 | 구름많음 | 구름많음 | 구름많음 |\n| 최저/최고기온 (℃) | -1℃ / 8℃ | -1℃ / 3℃ | -7℃ / -2℃ |\n| 강수확률 | 0% / 20% | 20% / 20% \n\n#### 대기오염 상세\n\n대기오염 상세 표 : 항목(오늘예보, 내일예보, 모레예보), 미세먼지(PM10), 초미세먼지(PM2.5)\n\n| 항목 | 오늘예보 | 내일예보 | 모레예보 |\n| 미세먼지 (PM10) | 나쁨 | 좋음 | 좋음 |\n| 초미세먼지 (PM2.5) | 나쁨 | 좋음 | 좋음 |\n\n#### 주간예보\n\n주간예보 표 : 항목별 날짜, 날씨, 최저/최고기온(℃) [...] | 항목 | 2025-12-01 | 2025-12-02 | 2025-12-03 | 2025-12-04 | 2025-12-05 |\n| 날씨 | 구름많음 | 구름많음 | 구름많음 | 구름많고 한때 비/눈 | 맑음 |\n| 최저/최고기온 (℃) | -1℃ / 8℃ | -1℃ / 3℃ | -7℃ / -2℃ | -7℃ / 4℃ | -4℃ / 4℃ |\n\n제공 : 케이웨더(Kweather) [...] Family Site; "패밀리사이트 메뉴 열기") \n\n 파트너스\n STO 소개\n 의료관광\n 서울관광 아카이브\n 서울관광플라자\n 디스커버서울패스\n 서울스테이\n 다누림관광\n 서울등산관광\n 서울컨벤션뷰로\n 서울컬쳐라운지\n Visit Seoul API\n SEOU

### example 2 : 맛집 추천 검색


In [20]:
result2 = agent_executor.invoke({"input": "강동구 명일역 최신 맛집 3개 추천해줘"})
print("\n" + "="*50)
print("최종 답변:")
print("="*50)
print(result2["output"])




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '강동구 명일역 최신 맛집 2023'}`


[{'title': '명일역 맛집 옹가네 한상차림 가성비 좋은 명일동 한정식 : 네이버 블로그', 'url': 'https://blog.naver.com/lse1507/223426635983', 'content': '2023년 11월 방문 당시. 2인 이상 주문 가능한 옹가네 한상차림 손맛 정식 외. 명품나물비빔밥 & 제육볶음. 된장찌개 & 순두부찌개 & 김치찌개 정식', 'score': 0.70847535}, {'title': '명일동 신상 맛집 카츠코이 바삭한 일식 돈까스의 진수 - 네이버 블로그', 'url': 'https://m.blog.naver.com/withsharp-/223669757367', 'content': '14:30, 19:30 (라스트오더)\n\n전화번호 / 0507-1410-6992\n\n서울특별시 강동구 상암로47길 80-17 1층 2호\n\n\u200b\n\n명일동 명일역 근처에 핫한곳들이 모여있는데\n\n명리단길이라고 부르더라고요ㅋ\n\n그 대각선 맞은편쪽 이구요\n\n명일전통시장 근처에 위치해 있습니다.\n\n\u200b\n\n\u200b\n\n가츠코이 매장 외부\n\n\u200b\n\n일식 돈까스 전문점 이라고 써있네용\n\n기대됩니다.\n\n\u200b\n\n\u200b\n\n가츠코이 매장 내부\n\n\u200b\n\n나오면서 찍어서 전체를 다 못찍었어요\n\n내부는 넓지는 않고 바테이블과\n\n2~4인석 테이블이 3개정도 있었구요\n\n일식집 느낌의 인테리어 입니다.\n\n딱 두자리가 있어서 저는 바로 앉을수 있었어요\n\n\u200b\n\n메뉴\n\n\u200b\n\n주문은 키오스크로 가능했고요\n\n메뉴는 돈까스와 카레로 간단했습니다.\n\n\u200b\n\n사실 치즈카츠를 먹으러 온거였는데\n\n이미 솔

### example 3 : 최신 뉴스 요약


In [21]:
result3 = agent_executor.invoke({"input": "Unity 최근 뉴스 요약해줘"})
print("\n" + "="*50)
print("최종 답변:")
print("="*50)
print(result3["output"])




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Unity 최근 뉴스'}`


[{'title': 'News - Unity Technologies - Investor Relations', 'url': 'https://investors.unity.com/news/default.aspx', 'content': 'Download, Unity Reports Third Quarter 2025 Financial Results, November 5, 2025, (opens in new window)\n\n October 22, 2025 \n\nUnity Launches Native, Cross-Platform Commerce Management for Game Developers Worldwide\n\nRead More\n\n   Download, Unity Launches Native, Cross-Platform Commerce Management for Game Developers Worldwide, October 22, 2025, (opens in new window)\n\n October 21, 2025 \n\nUnity Powers Launch Titles for Samsung’s Galaxy XR And Accelerates the Android XR Ecosystem\n\nRead More [...] Download, Unity Forms AI Council to Accelerate AI Product Innovation, September 25, 2025, (opens in new window)\n\n August 27, 2025 \n\nUnity Makes Ad Quality Tool Free to Help Improve Ad Experience and Player Retention\n\nRead More\n\n   Download,

### example 4 : 추가 질문


In [22]:
# 여기에 원하는 질문을 입력하세요
custom_question = "최근 AI 기술 동향에 대해 알려줘"

result_custom = agent_executor.invoke({"input": custom_question})
print("\n" + "="*50)
print("최종 답변:")
print("="*50)
print(result_custom["output"])




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '최근 AI 기술 동향'}`


[{'title': '인공지능 분야 산업·기술 동향 및 이슈', 'url': 'https://ettrends.etri.re.kr/ettrends/212/0905212008/0905212008.html', 'content': 'AI 기술의 발전 트렌드는 대표적으로 멀티모달 모델의 등장, AI의 현실 영향력 강화, 온디바이스 AI와 저전력‧경량 모델, 범용인공지능과 대안적 아키텍처 개발 등으로 정리해 볼 수 있다. 본고에서는 AI 기술의 발전 트렌드에 따라 산업‧기술 동향을 정리하였다.\n\n가. 멀티모달(Multimodal) [...] of reality and excessive resource consumption. Representative trends in AI technology development include the emergence of multimodal models, the strengthening of AI҆s grasp of reality, on-device AI, low-power and lightweight models, and the development of artificial general intelligence and alternative architectures. In addition, with the development of AI technology, its use in industry and daily life is gradually expanding. Various AI services are emerging, and AI technology is being applied [...] 본고에서는 AI 기술의 발전에 따른 산업 동향을 살펴보고, AI 산업의 주요 이슈인 AI 안전과 AI 지속가능성에 대해 소개하였다. AI 기